In [2]:
import pandas as pd
import numpy as np
import requests
import os
import yaml
import gzip
import random
from extractor import DataExtractor

In [21]:
extract_data = DataExtractor()
files = extract_data.get_data()

In [22]:
files

{'header':             0            1          2           3               4   \
 0  marketplace  customer_id  review_id  product_id  product_parent   
 
               5                 6            7              8            9   \
 0  product_title  product_category  star_rating  helpful_votes  total_votes   
 
      10                 11               12           13             14  
 0  vine  verified_purchase  review_headline  review_body  review_date\n  ,
 'train_pos':        0         1               2           3          4   \
 0      US    861463  R1HVYBSKLQJI5S  B00IX1I3G6  926539283   
 1      US    397970   RNYLPX611NB7Q  B005ESMGV4  379368939   
 2      US  22484620  R3R8PHAVJFTPDF  B004LLIKVU  473048287   
 3      US  14765851  R18WWEK8OIXE30  BT00CTP2EE  775486538   
 4      US   3559726   R6JH7A117FHFA  B004LLIKVU  473048287   
 ...    ..       ...             ...         ...        ...   
 97867  US  26443276  R2UN7Q20AQXJYG  B0002CZPPG  867256265   
 97868  US  4038

In [23]:
import csv
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
filename= url.split("/")[-1].replace('.',"_")
filepath= os.path.join(os.getcwd(),'data/',filename+'.gz')
header_filepath = os.path.join(os.getcwd(), 'data/header.tsv')

with gzip.open(filepath, "rt") as f, \
    open(header_filepath, 'w') as header:
        header_writer = csv.writer(header, delimiter='\t', lineterminator='\n')
        header_writer.writerow(f.readline().split('\t'))





In [ ]:
df= pd.read_csv('/app/data/amazon_reviews_us_Beauty_v1_00_tsv_gz.gz',compression='gzip',index_col=None,header=0, sep='\t', lineterminator='\r')
print(len(df))
print(df.size)

In [2]:
DataExtractor().split_train_test_samples(70,123)


In [4]:
import pandas as pd
df = pd.read_csv('/app/data/test_neg.tsv', header=None, sep="\t", error_bad_lines=False)


In [5]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,US,10278216,R1CJGF6M3PVHEZ,B001KYQA1S,9612905,"Maybelline Great Lash Washable Mascara, Clear ...",Beauty,1,0,2,N,Y,but it's like having nothing on them at all,I thought it would darken the lashes even thou...,2015-08-31\n
1,US,294612,R3N77VRU8Q2C,B00YQK1LVC,91155179,Premium Makeup Sponge Blender Duo – Black and ...,Beauty,1,0,0,N,Y,Was extremely disappointed in the size of the ...,Was extremely disappointed in the size of the ...,2015-08-31\n
2,US,3084759,R1FQS36HJASSOD,B007G4TVI0,400720811,OPTP Original McKenzie Cervical Roll - (703) P...,Beauty,1,0,1,N,Y,Don't really like it. Often makes my neck hurt...,Don't really like it. Often makes my neck hur...,2015-08-31\n
3,US,126519,R3MZIYP01O21NO,B00M4OB3ME,178823905,"Onedor 20"" Curly Full Head Clip in Synthetic H...",Beauty,2,0,0,N,Y,It's pretty soft and full but it knotted a lot...,It's pretty soft and full but it knotted a lot...,2015-08-31\n
4,US,24747434,RVH8BV8TGGHK5,B003ZS1O68,988287414,"Ardell Fashion Lashes Pair - 120 Demi, Brown (...",Beauty,1,0,0,N,Y,One Star,"not so nice, fake looking",2015-08-31\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216335,US,49201559,R172VWZ7YFD2L8,B000050B6W,977538732,Conair SS6 Straightening Iron with Mist,Beauty,1,9,9,N,N,"There are much better, cheaper irons out there","I have very thick, wavy hair, and this iron di...",2001-02-12\n
216336,US,48080659,R1O1A2GVJ6Y7NX,B000050FEP,105750022,HoMedics BMAT-2 Bubble Spa Bath Mat Massager w...,Beauty,1,5,6,N,N,Another HoMedics good idea but horrible implem...,"HoMedics have great ideas, but they rarely put...",2001-02-05\n
216337,US,52894048,R3OS9939ADAFXL,B000050FEP,105750022,HoMedics BMAT-2 Bubble Spa Bath Mat Massager w...,Beauty,1,3,3,N,N,Very disappointed,I was so looking forward to my new Bubblespa.....,2001-01-09\n
216338,US,48118327,RSBGH9O1NIUH6,B000050FEO,4337406,HoMedics Bubble Spa Massaging Bubble Mat,Beauty,2,9,9,N,N,Bubbles are Nice and relaxing.,The bubbles are nice and since my husband and ...,2001-01-09\n


In [5]:
from memory_profiler import memory_usage
def read_train_pos():
    return pd.read_csv('/app/data/train_pos.csv', header=None, sep="\t", error_bad_lines=False)


In [ ]:
mem_usage = memory_usage((read_train_pos(), ), timestamps=True, interval=1, multiprocess=True)
mem_usage

b'Skipping line 5665: expected 15 fields, saw 22\nSkipping line 17846: expected 15 fields, saw 22\nSkipping line 27876: expected 15 fields, saw 22\nSkipping line 42072: expected 15 fields, saw 22\nSkipping line 51627: expected 15 fields, saw 22\n'
b'Skipping line 88207: expected 15 fields, saw 22\nSkipping line 101697: expected 15 fields, saw 22\nSkipping line 108903: expected 15 fields, saw 22\nSkipping line 109492: expected 15 fields, saw 22\nSkipping line 121136: expected 15 fields, saw 22\n'
b'Skipping line 145215: expected 15 fields, saw 22\nSkipping line 147787: expected 15 fields, saw 22\nSkipping line 154213: expected 15 fields, saw 22\nSkipping line 155493: expected 15 fields, saw 22\nSkipping line 164423: expected 15 fields, saw 22\nSkipping line 187560: expected 15 fields, saw 22\n'
b'Skipping line 196798: expected 15 fields, saw 22\nSkipping line 203024: expected 15 fields, saw 22\nSkipping line 203749: expected 15 fields, saw 22\n'
b'Skipping line 268066: expected 15 field

In [23]:
import dask.dataframe as dd
from dask import delayed


def read_csv_dask():
    return dd.read_csv('/app/data/train_pos.tsv', header=None, sep="\t", error_bad_lines=False)

read_csv_dask().head(50)

/usr/local/lib/python3.8/site-packages/dask/dataframe/core.py:6560: UserWarning: Insufficient elements for `head`. 50 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date


In [14]:
import dask.dataframe as dd
df = dd.read_csv('/app/data/train_pos.tsv', header=None, sep="\t", error_bad_lines=False)
df.tail()

ParserError: Error tokenizing data. C error: EOF inside string starting at row 698

In [29]:
from datetime import datetime
data_type = [str, int, str, str, int, str, str, float, float, float, str, str, str,str, str]

In [ ]:
def split_train_test_samples(self, sample_size: float, seed: int):
    """
    creates a csv file and stores in the location specified
    :param sample_size: probability of selecting a sample and should be between 0 and 100
    :param seed: This is to assign random seed for reproducability
    :return: returns a pandas dataframe
    """
    # if sample_size<0 | sample_size> 100:
    #   raise Exception("Sample size need to be between 0 and 100")

    combined_filepath = self.get_filepath()

    with gzip.open(combined_filepath[0], 'rt', encoding="utf8") as f, \
            open(combined_filepath[1], 'w') as header, \
            open(combined_filepath[2], 'w') as train_pos, \
            open(combined_filepath[3], 'w') as train_neg, \
            open(combined_filepath[4], 'w') as test_pos, \
            open(combined_filepath[5], 'w') as test_neg:

        header.write(f.readline())
        random.seed(seed)
        for line in f:
            fields = line.split('\t')
            polarity = -1 if int(fields[7]) < 3 else 1 if int(fields[7]) > 3 else 0
            sample = random.choices([0, 1], weights=[100 - sample_size, sample_size], k=1)
            if sample[0] == 1:
                if polarity == 1:
                    train_pos.write(line)
                elif polarity == -1:
                    train_neg.write(line)
            else:
                if polarity == 1:
                    test_pos.write(line)
                elif polarity == -1:
                    test_neg.write(line)

In [23]:
from datetime import datetime
with open('/app/data/train_neg.tsv', 'r') as f:
    t = f.readline()
    t = t.split('\t')
    if len(t) == 15:    
        try:
            new_line = []
            for i in range(15):
                new_format = None
                if i in [0,2,3,5,6,10,11,12,13]:
                    new_format = str(t[i])
                elif i in [1,4,7,8,9]:
                    new_format = int(t[i])
                elif i == 14:
                    new_format = datetime.strptime(t[14].split('\n')[0], '%Y-%m-%d')
                new_line.append(new_format)
            if len(new_line) == 15:
                df = pd.DataFrame(new_line, header=None)
        except ValueError:
            print('dodgy data')

print(t)    
print(new_line)
print(df)


dodgy data
['US', '41137196', 'R1O5HZBOVLX8CY', 'B006MWL2ZM', '57378789', 'Goddess Remi Human Hair Weave Bump - Trio 2 4 6', 'Beauty', '2', '0', '0', 'N', 'Y', 'Two Stars', 'This was not what I expected.', '"2015-08-31\n']
['US', 41137196, 'R1O5HZBOVLX8CY', 'B006MWL2ZM', 57378789, 'Goddess Remi Human Hair Weave Bump - Trio 2 4 6', 'Beauty', 2, 0, 0, 'N', 'Y', 'Two Stars', 'This was not what I expected.']
Dask DataFrame Structure:
                    0        1       2       3        4       5       6        7        8        9       10      11      12      13      14
npartitions=23                                                                                                                             
                object  float64  object  object  float64  object  object  float64  float64  float64  object  object  object  object  object
                   ...      ...     ...     ...      ...     ...     ...      ...      ...      ...     ...     ...     ...     ...     ...
...   

In [23]:
import csv
with open('/app/data/test_neg.csv', 'r') as f:
    badline = 0
    goodline = 0
    writer = csv.writer(f, delimiter = '\t', newline = '\n')
    for line in f:
        t = line.split('\t')
        if len(t) == 15:    
            try:
                new_line
                for i in range(15):
                    new_format = None
                    if i in [0,2,3,5,6,10,11,12,13]:
                        new_format = str(t[i])
                    elif i in [1,4,7,8,9]:
                        new_format = int(t[i])
                    elif i == 14:
                        new_format = datetime.strptime(t[14].split('\n')[0], '%Y-%m-%d')
                    new_line.append(new_format)
            except: 
                badline += 1
            
            
            
                        
                
            try:
                t[]
            goodline +=1
        else:
            badline +=1
print(goodline,badline)
        

2797569 0


In [14]:
list(range(15))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [1]:
from utils.utils import TextProcessing

text = "this is america and its beautiful."
TextProcessing().process_sentence(sentence=text)

['america', 'beautiful', '.']

In [15]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [24]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
word_clean = []
for word in tok:
    word_clean.extend(lemmatizer(word))
word

In [ ]:
def word_label_count():
    with open('train_pos', 'r') as file:
        freqs = {}
        for line in file:
            text = line.split("\t")[7]
            for word in process_sentence(text):
                pair = (word,y)
                freqs[pair] = freqs.get(pair,0) + 1
                